<a href="https://colab.research.google.com/github/vatiza/Machine_Learning/blob/main/Unboxing_an_Industry_Level_Deep_Learning_Model_for_Brain_Anomaly_Detection_An_Elaborative_Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect Kaggle APi**

In [3]:
! pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vatiza","key":"fa2cb6786279d6030614ac8885a33cb1"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

***Import Brain Tumor Mri Dataset From `kaggle` ***

Dataset location

[Brain Tumor MRi Dataset](https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset?datasetId=1608934)

In [9]:
! kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset


brain-tumor-mri-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
! unzip brain-tumor-mri-dataset.zip -d train/

In [37]:
# import library
import tensorflow as tf
import os
import cv2
import imghdr
from matplotlib import pyplot as plt
import matplotlib.image as mpimg


In [21]:
data='/content/brain-tumor-mri-dataset'
img_exts=['jpg']

In [33]:
img=cv2.imread(os.path.join('data','notumor','Te-no_0391.jpg',0))

In [27]:
os.listdir(os.path.join(data,'Testing','notumor'))


['Te-no_0018.jpg',
 'Te-no_0360.jpg',
 'Te-no_0371.jpg',
 'Te-no_0066.jpg',
 'Te-no_0213.jpg',
 'Te-no_0144.jpg',
 'Te-no_0036.jpg',
 'Te-no_0385.jpg',
 'Te-no_0158.jpg',
 'Te-no_0215.jpg',
 'Te-no_0253.jpg',
 'Te-no_0401.jpg',
 'Te-no_0285.jpg',
 'Te-no_0156.jpg',
 'Te-no_0068.jpg',
 'Te-no_0212.jpg',
 'Te-no_0390.jpg',
 'Te-no_0269.jpg',
 'Te-no_0091.jpg',
 'Te-no_0135.jpg',
 'Te-no_0356.jpg',
 'Te-no_0326.jpg',
 'Te-no_0293.jpg',
 'Te-no_0240.jpg',
 'Te-no_0388.jpg',
 'Te-no_0396.jpg',
 'Te-no_0058.jpg',
 'Te-no_0111.jpg',
 'Te-no_0164.jpg',
 'Te-no_0171.jpg',
 'Te-no_0149.jpg',
 'Te-no_0055.jpg',
 'Te-no_0210.jpg',
 'Te-no_0225.jpg',
 'Te-no_0079.jpg',
 'Te-no_0373.jpg',
 'Te-noTr_0006.jpg',
 'Te-no_0398.jpg',
 'Te-no_0346.jpg',
 'Te-no_0178.jpg',
 'Te-no_0341.jpg',
 'Te-no_0227.jpg',
 'Te-no_0108.jpg',
 'Te-no_0376.jpg',
 'Te-no_0044.jpg',
 'Te-no_0383.jpg',
 'Te-no_0281.jpg',
 'Te-no_0291.jpg',
 'Te-no_0142.jpg',
 'Te-no_0094.jpg',
 'Te-no_0332.jpg',
 'Te-no_0196.jpg',
 'Te-no_02